In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from glob import iglob
from math import log, floor
import os
import re
import pickle

%reload_ext autoreload
%load_ext blackcellmagic
%autoreload 2

from lib.plot_utils import *
from lib.cdf_utils import *
from lib.scatter_utils import *

## Scatter Plot of Total Flow

In [ ]:
def pop_random_scatter(path_form_csv, pop_csv, query_str):
    with_kdl = "problem == 'Kdl.graphml'"
    no_kdl = "problem != 'Kdl.graphml'"
    ratio_dfs = get_ratio_dataframes(
        path_form_csv,
        pop_csv,
        pop_parent_query_str=query_str,
        pop_query_strs=[with_kdl, no_kdl],
    )
    plot_dfs = [ratio_dfs["POP"][with_kdl], ratio_dfs["POP"][no_kdl]]
    scatter_plot(
        plot_dfs,
        ["KDL", "Non-KDL"],
        ["nc", "smore"],
        "speedup_ratio",
        "obj_val_ratio",
        xlim=None,
        ylim=[0.0, 1.1],
        figsize=(7.5, 4),
        xlabel="Speedup rel. to original problem",
        ylabel="Total flow rel. to\n original problem",
        bbta=(0, 0, 1, 2.35),
        ncol=2,
        xlog=True,
        save=True,
    )

In [ ]:
path_form_csv = 'max_total_flow_csvs/path-form-total-flow-slices_0_1_2_3_4.csv'
pop_csv = 'max_total_flow_csvs/pop-total_flow-slice_0-k_16.csv'

In [ ]:
pop_random_scatter(
    path_form_csv,
    pop_csv,
    "split_method == 'random' and num_subproblems == 16 and (tm_model == 'poisson-high-intra' and split_fraction == 0.75) or (tm_model != 'poisson-high-intra' and split_fraction == 0.0)",
)

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'poisson-high-inter'", "Poisson, High Inter")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'uniform'", "Uniform")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'gravity'", "Gravity")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'bimodal'", "Bimodal")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   None, "all")